Intake-STAC + Planetscope
================

In this notebook, we'll take a look at some of the functionality in Intake-STAC by exploring STAC catalogs such as the open-access [Planet Disaster data catalog](https://planet.stac.cloud/?t=catalogs) hosted at https://raw.githubusercontent.com/cholmes/sample-stac/master/stac/catalog.json.

STAC metadata is organized into a hierarchy of **Catalogs, Collections, and Items**. Items ultimately hold *Assets* which are data files such as satellite raster images. Ultimately the goal of intake-STAC is to facilitate loading these *Assets* directly into Python objects for interactive computation without worrying about filepaths and URLs.

In [ ]:
%matplotlib inline
import intake

In [ ]:
# intake checks for registered drivers when imported
# You should see 'stac_catalog, stac_collection, stac_item, and stac_item_collection' if intake-stac is installed
list(intake.registry)

## Catalog adapter

We'll start by connecting to the STAC Catalog for `planet-disaster-data`. We read the top-level `catalog.json` file and drill down until we get to specific STAC Items.

In [ ]:
# Load root catalog
url = 'https://raw.githubusercontent.com/cholmes/sample-stac/master/stac/catalog.json'
cat = intake.open_stac_catalog(url)
print(cat.name)

# This lists available subcatalogs:
list(cat)

In [ ]:
# STAC files are in JSON format, which is accessible as Python dictionaries:
cat.metadata

In [ ]:
# Drill down into subcatalogs
subcat = cat['hurricane-harvey']
list(subcat)

In [ ]:
# Another subcatalog!
subcat1 = subcat['hurricane-harvey-0831']
list(subcat1)

In [ ]:
# Load a STAC Item
item = subcat1['Houston-East-20170831-103f-100d-0f4f-RGB']

In [ ]:
item.metadata

In [ ]:
# Item Assets are accessible via lists just like subcatalogs:
list(item)

In [ ]:
item['thumbnail'].metadata

In [ ]:
# Finally we can display an image!
from IPython.display import Image
Image(item['thumbnail'].urlpath)

In [ ]:
# Or we can load the image directly into Xarray for analysis
# The full resolution image is big, but we use dask behind the scenes to only read metadata at first
asset = item['mosaic']
da = asset.to_dask()
da

In [ ]:
# The chunk structure isn't set by default
# setting a chunk structure makes operating on subsets of pixels more efficient
da = da.chunk(dict(band=1, x=2560, y=2560))
da

In [ ]:
da.isel(band=0, x=slice(0, 2560), y=slice(0, 1280)).plot.imshow()